In [5]:
#!/usr/bin/env python

import numpy as np
import jax
import jax.numpy as npj

init_coord = []
init_vel = []

with open('input.txt') as f:
    contents = f.readlines()
    N_of_Atoms = int(contents[0])
    side_legth = float(contents[2])
    for i in range(N_of_Atoms):
        Inp = contents[3+i].split()
        coord = [float(e) for e in Inp[:3]]
        vel = [float(e) for e in Inp[3:]]
        init_coord.append(coord)
        init_vel.append(vel)
coordinates = npj.array(init_coord)
        
#print (N_of_Atoms)
print (f"init_coord: {init_coord}")
  

def E_pot(positions):
    delta = positions[:, npj.newaxis, :] - positions
    indices = npj.triu_indices(positions.shape[0], k=1)
    delta = delta[indices[0], indices[1], :]
    r = npj.sqrt((delta * delta).sum(axis=1))
    print (f"r: {r}")
    D_e = 1.6
    Alpha = 3.028
    r_e = 1.411
    V_Morse = D_e * (npj.exp(-2*Alpha*(r-r_e))-2*npj.exp(-Alpha*(r-r_e)))
    E_pot = npj.sum(V_Morse)
    return E_pot

print (f"E_pot: {E_pot(coordinates)}")

morse_gradient = jax.jit(jax.grad(E_pot))

e_pot = E_pot(coordinates)
forces = - morse_gradient(coordinates)

print("Potential energy:", e_pot, "Forces:", forces, sep="\n")


init_coord: [[0.1, 0.5, 0.9], [5.0, 6.0, 8.0], [1.0, 2.0, 3.0], [1.0, 0.0, 6.0]]
r: [10.230836   2.73313    5.2028837  7.5498343  7.483315   3.6055512]
E_pot: -0.062069009989500046
r: [10.230836   2.73313    5.2028837  7.5498343  7.483315   3.6055512]
r: Traced<ShapedArray(float32[6])>with<JVPTrace(level=2/1)> with
  primal = Traced<ShapedArray(float32[6])>with<DynamicJaxprTrace(level=0/1)>
  tangent = Traced<ShapedArray(float32[6])>with<JaxprTrace(level=1/1)> with
    pval = (ShapedArray(float32[6]), *)
    recipe = JaxprEqnRecipe(eqn_id=<object object at 0x10e537850>, invars=(Traced<ShapedArray(float32[6]):JaxprTrace(level=1/1)>, Traced<ShapedArray(float32[6]):JaxprTrace(level=1/1)>), outvars=[<weakref at 0x132349290; to 'JaxprTracer' at 0x1322ecd10>], primitive=xla_call, params={'device': None, 'backend': None, 'name': 'jvp(<lambda>)', 'donated_invars': (False, False), 'inline': True, 'call_jaxpr': { lambda ; a:f32[6] b:f32[6]. let c:f32[6] = mul b a in (c,) }}, effects=set(), sourc